In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv("C:/Users/ANJALIKRISHNA/Desktop/NetAegis/Ml/cleaned_network_data.csv")
df.head()

,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,...,dns_rcode,dns_AA,dns_RD,dns_RA,dns_rejected,http_request_body_len,http_response_body_len,http_status_code,label,type
0,192.168.1.37,4444,192.168.1.193,49178,tcp,-,290.371539,101568,2592,OTH,...,0,none,none,none,none,0,0,0,1,backdoor
1,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000102,0,0,REJ,...,0,none,none,none,none,0,0,0,1,backdoor
2,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000148,0,0,REJ,...,0,none,none,none,none,0,0,0,1,backdoor
3,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000113,0,0,REJ,...,0,none,none,none,none,0,0,0,1,backdoor
4,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000130,0,0,REJ,...,0,none,none,none,none,0,0,0,1,backdoor


In [11]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
# Use 'type' as your target column
y_raw = df['type']

# Encode labels into numbers
le = LabelEncoder()
y = le.fit_transform(y_raw)
unique, counts = np.unique(y, return_counts=True)
for label, count in zip(le.inverse_transform(unique), counts):
    print(f"Class '{label}' has {count} samples")


Class 'backdoor' has 18711 samples
Class 'ddos' has 19993 samples
Class 'dos' has 18992 samples
Class 'injection' has 19964 samples
Class 'mitm' has 1041 samples
Class 'normal' has 42021 samples
Class 'password' has 19861 samples
Class 'ransomware' has 14735 samples
Class 'scanning' has 20000 samples
Class 'xss' has 15137 samples


In [12]:
# Find and remove rows where the original 'type' column is NaN or 'nan'
df = df[df['type'].notna()]                    # Remove actual NaNs
df = df[df['type'].str.lower() != 'nan']       # Remove string 'nan' if present


In [13]:

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Drop columns that are not useful for training and are not present in the dataframe
columns_to_drop = ['Flow ID', 'Source IP', 'Destination IP', 'Timestamp']
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
df = df.drop(existing_columns_to_drop, axis=1)

# Separate features (X) and target (y)
X = df.drop('type', axis=1)
y = df['type']

# Convert target variable to numerical labels if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)



In [14]:
# Convert object columns to category dtype
for col in X.select_dtypes(include='object').columns:
    X[col] = X[col].astype('category')


In [15]:
# Split again after conversion
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Re-initialize and train model
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', enable_categorical=True)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


Accuracy: 0.9967971436822346
Classification Report:
              precision    recall  f1-score   support

    backdoor       1.00      1.00      1.00      3742
        ddos       1.00      0.99      1.00      3999
         dos       1.00      0.99      0.99      3798
   injection       0.99      0.99      0.99      3993
        mitm       0.94      0.95      0.94       208
      normal       1.00      1.00      1.00      8404
    password       1.00      1.00      1.00      3972
  ransomware       1.00      1.00      1.00      2947
    scanning       0.99      0.99      0.99      4000
         xss       1.00      1.00      1.00      3028

    accuracy                           1.00     38091
   macro avg       0.99      0.99      0.99     38091
weighted avg       1.00      1.00      1.00     38091



In [19]:
import joblib

# Save the trained model
joblib.dump(model, 'xgboost_model.pkl')


['xgboost_model.pkl']

In [20]:
import pickle

with open("xgboost_model.pkl", "wb") as f:
    pickle.dump(model, f)


In [22]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier


# Encode object or category columns
for col in X.select_dtypes(include=['object', 'category']).columns:
    le_col = LabelEncoder()
    X[col] = le_col.fit_transform(X[col])

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train RandomForest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)




Accuracy: 0.9959045443805623
Classification Report:
              precision    recall  f1-score   support

    backdoor       1.00      1.00      1.00      3742
        ddos       0.99      0.99      0.99      3999
         dos       0.99      0.99      0.99      3798
   injection       0.99      0.99      0.99      3993
        mitm       0.93      0.91      0.92       208
      normal       1.00      1.00      1.00      8404
    password       1.00      1.00      1.00      3972
  ransomware       1.00      1.00      1.00      2947
    scanning       0.99      0.99      0.99      4000
         xss       1.00      1.00      1.00      3028

    accuracy                           1.00     38091
   macro avg       0.99      0.99      0.99     38091
weighted avg       1.00      1.00      1.00     38091



In [24]:
# Save model
joblib.dump(rf_model, 'random_forest_model.pkl')

import pickle

with open("random_forest_model.pkl", "wb") as f:
    pickle.dump(model, f)



In [25]:
# Get predictions
xgb_preds = model.predict(X_test)
rf_preds = rf_model.predict(X_test)

# Decode back to class labels
xgb_labels = le.inverse_transform(xgb_preds)
rf_labels = le.inverse_transform(rf_preds)
true_labels = le.inverse_transform(y_test)

# Create a comparison DataFrame
import pandas as pd

comparison_df = pd.DataFrame({
    'True Label': true_labels,
    'XGBoost Prediction': xgb_labels,
    'Random Forest Prediction': rf_labels
})

# Show the first few rows
print(comparison_df.head(20))


   True Label XGBoost Prediction Random Forest Prediction
0      normal             normal                   normal
1        ddos               ddos                     ddos
2        mitm               mitm                     mitm
3        ddos               ddos                     ddos
4    backdoor           backdoor                 backdoor
5      normal             normal                   normal
6        ddos               ddos                     ddos
7      normal             normal                   normal
8   injection          injection                injection
9    password           password                 password
10   scanning           scanning                 scanning
11   scanning           scanning                 scanning
12     normal             normal                   normal
13        dos                dos                      dos
14  injection          injection                injection
15     normal             normal                   normal
16        xss 